In [ ]:
import numpy as np

# Load the .npy file
# pointcloud_data = np.load('/Users/stevenymchang/Downloads/slam01_laneline_label.npy')
pointcloud_data = np.load(r"C:\Users\Bryan\Desktop\ece484\Final_Project\ece484_final\mapping_test\slam07_mapping_local.npy")

# Print the shape of the data (frames, channels, points, attributes)
print("Shape of the point cloud data:", pointcloud_data.shape)

Shape of the point cloud data: (632, 128, 1024, 7)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from kiss_icp.pipeline import OdometryPipeline
from dataclasses import dataclass
from typing import Dict, List

# Create dataset class similar to their structure
class CustomDataset:
    def __init__(self, data):
        """
        data: numpy array of shape (632, 128, 1024, 7)
        """
        self.data = data
        self.length = len(data)
        self.sequence_id = "custom_sequence"  # Required by pipeline
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, idx):
        """Returns frame at index idx"""
        frame = self.data[idx, :, :, :3].reshape(-1, 3)  # Only take xyz coordinates
        timestamps = np.zeros(len(frame))  # Dummy timestamps
        return frame, timestamps

@dataclass
class Metric:
    units: str
    values: List

def run_kiss_icp(data):
    # Create results dictionary
    results = {}
    
    # Create dataset
    dataset = CustomDataset(data)
    
    # Create pipeline
    pipeline = OdometryPipeline(
        dataset=dataset,
        deskew=False,
        max_range=100.0,
        visualize=False
    )
    
    # Run pipeline
    print(f"Processing sequence...")
    seq_res = pipeline.run()
    seq_res.print()
    
    # Get poses and transform points
    poses = np.asarray(pipeline.poses)
    
    # Transform all frames to global coordinate system
    all_global_points = []
    for frame_idx in range(len(dataset)):
        # Get current frame
        current_frame, _ = dataset[frame_idx]
        
        # Transform points using pose
        global_points = transform_points(current_frame, poses[frame_idx])
        all_global_points.append(global_points)
    
    # Combine all transformed points
    global_map = np.vstack(all_global_points)
    
    return global_map, poses

def transform_points(points, pose):
    """Transform points using pose matrix"""
    points_h = np.hstack([points, np.ones((len(points), 1))])
    global_points = (pose @ points_h.T).T
    return global_points[:, :3]

def plot_point_cloud(points):
    """Plot point cloud from different views"""
    fig = plt.figure(figsize=(15, 5))
    
    # Top view (XY plane)
    plt.subplot(131)
    plt.scatter(points[:, 0], points[:, 1],
               c='blue', s=1, alpha=0.1)
    plt.xlabel('X (m)')
    plt.ylabel('Y (m)')
    plt.title('Top View')
    plt.axis('equal')
    
    # Side view (XZ plane)
    plt.subplot(132)
    plt.scatter(points[:, 0], points[:, 2],
               c='blue', s=1, alpha=0.1)
    plt.xlabel('X (m)')
    plt.ylabel('Z (m)')
    plt.title('Side View')
    plt.axis('equal')
    
    # 3D view
    ax = plt.subplot(133, projection='3d')
    scatter = ax.scatter(points[:, 0], 
                        points[:, 1], 
                        points[:, 2],
                        c='blue', s=1, alpha=0.1)
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    plt.title('3D View')
    
    plt.tight_layout()
    plt.show()

# Usage
if __name__ == "__main__":
    # Load data
    data = np.load(r"C:\Users\Bryan\Desktop\ece484\Final_Project\ece484_final\mapping_test\slam07_mapping_local.npy")  # (632, 128, 1024, 7)
    
    # Run KISS-ICP and get transformed points
    global_map, poses = run_kiss_icp(data)
    
    # Plot results
    plot_point_cloud(global_map)
    
    print(f"Processed {len(poses)} frames")
    print(f"Total points in global map: {len(global_map)}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from kiss_icp.pipeline import OdometryPipeline
from kiss_icp.config import KISSConfig

class CustomDataset:
    def __init__(self, data):
        """
        data: numpy array of shape (632, 128, 1024, 7)
        """
        self.data = data
        self.length = len(data)
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, idx):
        """Returns frame at index idx"""
        frame = self.data[idx, :, :, :3].reshape(-1, 3)  # Only take xyz coordinates
        timestamps = np.zeros(len(frame))  # Dummy timestamps
        return frame, timestamps

class FrameMapper:
    def __init__(self, data):
        # Create dataset
        self.dataset = CustomDataset(data)
        
        # Initialize KISS-ICP pipeline
        self.pipeline = OdometryPipeline(
            dataset=self.dataset,
            deskew=False,
            max_range=100.0,
            visualize=False
        )
        
        self.reference_frame = None
        self.global_map = None
        self.poses = []

    def process_data(self):
        # Run the pipeline
        results = self.pipeline.run()
        
        # Get all poses
        self.poses = self.pipeline.poses
        
        # Process all frames with poses
        frames = len(self.dataset)
        all_global_points = []
        
        for frame_idx in range(frames):
            # Get current frame
            current_frame = self.dataset[frame_idx][0]  # [0] because __getitem__ returns (frame, timestamps)
            
            # Transform points using pose
            global_points = self.transform_points(current_frame, self.poses[frame_idx])
            all_global_points.append(global_points)
        
        # Combine all transformed points
        self.global_map = np.vstack(all_global_points)
        print(f"Processed {frames} frames")

    def transform_points(self, points, pose):
        points_h = np.hstack([points, np.ones((len(points), 1))])
        global_points = (pose @ points_h.T).T
        return global_points[:, :3]

    def plot_global_map(self):
        """Plot all transformed frames in one visualization"""
        fig = plt.figure(figsize=(15, 5))
        
        # Top view (XY plane)
        plt.subplot(131)
        plt.scatter(self.global_map[:, 0], 
                   self.global_map[:, 1],
                   c='blue', s=1, alpha=0.1)
        plt.xlabel('X (m)')
        plt.ylabel('Y (m)')
        plt.title('Top View - All Frames')
        plt.axis('equal')
        
        # Side view (XZ plane)
        plt.subplot(132)
        plt.scatter(self.global_map[:, 0], 
                   self.global_map[:, 2],
                   c='blue', s=1, alpha=0.1)
        plt.xlabel('X (m)')
        plt.ylabel('Z (m)')
        plt.title('Side View - All Frames')
        plt.axis('equal')
        
        # 3D view
        ax = plt.subplot(133, projection='3d')
        scatter = ax.scatter(self.global_map[:, 0], 
                           self.global_map[:, 1], 
                           self.global_map[:, 2],
                           c='blue', s=1, alpha=0.1)
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_zlabel('Z (m)')
        plt.title('3D View - All Frames')
        
        plt.tight_layout()
        plt.show()

# Usage
if __name__ == "__main__":
    # Load data
    data = np.load(r"C:\Users\Bryan\Desktop\ece484\Final_Project\ece484_final\mapping_test\slam07_mapping_local.npy") # (632, 128, 1024, 7)
    
    # Create mapper and process
    mapper = FrameMapper(data)
    mapper.process_data()
    
    # Plot all frames mapped together
    mapper.plot_global_map()
    
    # # Save results
    # np.savez("mapping_results.npz",
    #          reference_frame=mapper.reference_frame,
    #          global_map=mapper.global_map,
    #          poses=mapper.poses)


In [6]:
# converting data
import numpy as np
import os

# Load your data (shape: 408, 128, 1024, 7)
data = pointcloud_data 

# Directory for output
output_dir = r"D:\484_final_project\dataset\sequences\06\velodyne"
os.makedirs(output_dir, exist_ok=True)
max = 65535.0

# Iterate over each frame and process
for frame_idx in range(data.shape[0]):
    frame_data = data[frame_idx]  # Shape (128, 1024, 7)
    
    # Extract x, y, z, and near IR (attributes 0, 1, 2, and 6)
    points = frame_data[:, :, [0, 1, 2, 6]]
    
    # Normalize near IR (attribute 6) to be between 0 and 1
    near_ir = points[:, :, 3]
    points[:, :, 3] = near_ir / max
    
    # Flatten channels and points to get shape (131072, 4)
    points = points.reshape(-1, 4)
    
    # Ensure the data is of type float32 for compatibility
    points = points.astype(np.float32)
    
    # Save each frame to a .bin file
    output_path = os.path.join(output_dir, f"{str(frame_idx).zfill(6)}.bin")
    points.tofile(output_path)

print("Conversion complete.")

Conversion complete.


In [1]:
# converting label
import numpy as np
import os

# Load your labels data (shape: 407, 128, 1024)
labels = np.load(r"D:\484_final_project\processed_point_cloud\slam06_predictions.npy")

# Directory for output
output_dir = r"D:\484_final_project\dataset\sequences\06"
os.makedirs(output_dir, exist_ok=True)

# Iterate over each frame and flatten the channels and points
for frame_idx in range(labels.shape[0]):
    frame_labels = labels[frame_idx]  # Shape (128, 1024)
    
    # Flatten the (128, 1024) to (131072,) for each frame
    flattened_labels = frame_labels.reshape(-1)  # Shape (131072,)
    
    # Ensure labels are stored as integers
    flattened_labels = flattened_labels.astype(np.int32)
    
    # Save the flattened labels as a .bin file
    labels_output_path = os.path.join(output_dir, f"{str(frame_idx).zfill(6)}.label")
    flattened_labels.tofile(labels_output_path)

print("Label conversion complete.")

Label conversion complete.
